In [1]:
from ecell4 import *
%matplotlib inline
import numpy as np
import math
import sys
from math import sqrt
import matplotlib.pyplot as plt
from ecell4.extra import ensemble
import numpy as np
from math import pi

def wrap(job,job_id,task_id):
    def singlerun(kf,phit,phi,L,run,duration):
        import numpy as np
        rm=0.005
        D=1.
        R=2*rm
        rng = GSLRandomNumberGenerator()
        rng.seed(run)   
        f=1.0208582         
        rv=rm*f
        td =(2*rv)**2/(6*D)   
        kd = 4*math.pi*R*D*2
        kb=kf*kd
        with species_attributes():
            C | {'D':str(0.0),'radius':str(rv)}
            E | {'D':str(D),'radius':str(rv)}
            S | {'D':str(D),'radius':str(rv)}
            ES | {'D':str(0.0),'radius':str(rv)}
        with reaction_rules():
            #E+S >ES | kb
            E+S>~E+~S|kb


        m=get_model()
        w = spatiocyte.SpatiocyteWorld(Real3(L,L,L),rv,rng)
        w.bind_to(m)
        size=tuple(w.calculate_shape(Real3(L,L,L),rv))    
        Nev = size[0]*size[1]*size[2]
        NS=int(Nev*phit)
        NE=int(NS/5)
        Nc=int(Nev*phi)            
        w.add_molecules(Species('C'), Nc)
        w.add_molecules(Species('S'), NS)
        w.add_molecules(Species('E'), NE)
        sim = spatiocyte.SpatiocyteSimulator(w)
        sim.initialize()
        #duration = duration*td
        obs=FixedIntervalNumberObserver(td,['E','S'])
        sim.run(duration,obs)        
        #tlogs,nalog=np.array(obs.data()).T
        if run==1:
            print('kf={},phit={},phi={},L={}'.format(kf,phit,phi,L))
            #print('td={:.4e},kb={:.4e},kr={:.4e},kub={:.4e},kon={:.4e},Nc={},NE={},NS={},duration={:.4e}'.format(td,kb,kr,kub,kon,Nc,NE,NS,duration))
        return obs.data()#tlogs,nalog
    job.update({'run':task_id})
    out=singlerun(**job)
    return out

def exe(jobs,simrun):    
    #res=ensemble.run_multiprocessing(wrap,jobs,n=simrun,nproc=60,modules=('numpy','ecell4','math'))[0]
    res=ensemble.run_slurm(wrap,jobs,n=simrun,nproc=60,path='tmp',modules=('numpy','ecell4','math'),extra_args=['--exclude=ibc[02,12,27]'])[0]
    res = np.array(res)
    meanres = np.mean(res,axis=0)   
    tt=meanres.T[0]
    Ne=meanres.T[1]
    Ns=meanres.T[2]
    #Nes=meanres.T[3]
    name='/home/chew/outputdata/3Dirxncrowding_kf{:.1f}_phi{:.1f}phit{:.3f}L{}_Nratio5'.format(jobs[0]["kf"],jobs[0]["phi"],jobs[0]["phit"],jobs[0]["L"])
    f=open(name,'w')
    np.savetxt(name,np.column_stack([tt,Ns,Ne]),delimiter=',')
    f.close()    
    print(name,len(tt))
    plt.plot(tt,Ne,label='E')
    plt.semilogx(tt,Ns,label='S')
    #plt.plot(tt,Nes,label='ES')
    print('first',Ns[0],'last',Ns[-1])
    print('first',Ne[0],'last',Ne[-1])
    plt.legend()

In [4]:
kf=10.
phit=0.001
L=1.0
dur=0.02#1e4
simrun=1000
phis=[0.3]
for phi in phis:
    jobs = [{'kf':kf,'phit':phit,'phi':phi,'L':L,'duration':dur}]
    exe(jobs,simrun)

DEBUG:sge:Submitted batch job 3018326
DEBUG:sge:             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
DEBUG:sge:3018326_[1-1000%60     debug sge-j4o0     chew PD       0:00      1 (None)
DEBUG:sge:           2754894     debug job_08.s    niina  R 23-00:49:13      1 ibc01
DEBUG:sge:           2754895     debug job_09.s    niina  R 23-00:49:13      1 ibc01
DEBUG:sge:           2754897     debug job_16.s    niina  R 23-00:49:13      1 ibc01
DEBUG:sge:           2762398     debug plane_01    niina  R 22-22:49:46      1 ibc04
DEBUG:sge:           2762399     debug plane_02    niina  R 22-22:49:46      1 ibc04
DEBUG:sge:           2762400     debug plane_03    niina  R 22-22:49:46      1 ibc06
DEBUG:sge:           2762401     debug plane_04    niina  R 22-22:49:46      1 ibc06
DEBUG:sge:           2762402     debug plane_05    niina  R 22-22:49:46      1 ibc06
DEBUG:sge:           2762403     debug plane_06    niina  R 22-22:49:46      1 ibc19
DEBUG:sge:       

DEBUG:sge:           2765428     debug plane_31    niina  R 22-22:19:22      1 ibc19
DEBUG:sge:           2765429     debug plane_32    niina  R 22-22:19:22      1 ibc19
DEBUG:sge:           2765430     debug  re01.sh    niina  R 22-21:45:39      1 ibc19
DEBUG:sge:           2765431     debug  re08.sh    niina  R 22-21:45:33      1 ibc19
DEBUG:sge:           2765432     debug  re09.sh    niina  R 22-21:45:30      1 ibc19
DEBUG:sge:           2765433     debug  re14.sh    niina  R 22-21:45:27      1 ibc19
DEBUG:sge:           2765434     debug  re16.sh    niina  R 22-21:45:27      1 ibc19
DEBUG:sge:         3018238_1     debug sge-8qij     chew  R      23:30      1 ibc06
DEBUG:sge:         3018238_2     debug sge-8qij     chew  R      23:30      1 ibc06
DEBUG:sge:         3018238_3     debug sge-8qij     chew  R      23:30      1 ibc06
DEBUG:sge:         3018238_4     debug sge-8qij     chew  R      23:30      1 ibc06
DEBUG:sge:         3018238_5     debug sge-8qij     chew  R      23:3

DEBUG:sge:           2762402     debug plane_05    niina  R 22-22:50:06      1 ibc06
DEBUG:sge:           2762403     debug plane_06    niina  R 22-22:50:06      1 ibc19
DEBUG:sge:           2762404     debug plane_07    niina  R 22-22:50:06      1 ibc19
DEBUG:sge:           2762405     debug plane_08    niina  R 22-22:50:06      1 ibc19
DEBUG:sge:           2762406     debug plane_09    niina  R 22-22:50:06      1 ibc18
DEBUG:sge:           2762407     debug plane_10    niina  R 22-22:50:06      1 ibc18
DEBUG:sge:           2762408     debug plane_11    niina  R 22-22:50:06      1 ibc18
DEBUG:sge:           2762409     debug plane_12    niina  R 22-22:50:06      1 ibc18
DEBUG:sge:           2762411     debug plane_14    niina  R 22-22:50:06      1 ibc18
DEBUG:sge:           2762412     debug plane_15    niina  R 22-22:50:06      1 ibc18
DEBUG:sge:           2762413     debug plane_16    niina  R 22-22:50:06      1 ibc17
DEBUG:sge:           2765414     debug plane_17    niina  R 22-22

DEBUG:sge:        3018326_47     debug sge-j4o0     chew  R       0:20      1 ibc04
DEBUG:sge:        3018326_48     debug sge-j4o0     chew  R       0:20      1 ibc04
DEBUG:sge:        3018326_49     debug sge-j4o0     chew  R       0:20      1 ibc04
DEBUG:sge:        3018326_50     debug sge-j4o0     chew  R       0:20      1 ibc04
DEBUG:sge:        3018326_51     debug sge-j4o0     chew  R       0:20      1 ibc04
DEBUG:sge:        3018326_52     debug sge-j4o0     chew  R       0:20      1 ibc04
DEBUG:sge:        3018326_53     debug sge-j4o0     chew  R       0:20      1 ibc03
DEBUG:sge:        3018326_54     debug sge-j4o0     chew  R       0:20      1 ibc03
DEBUG:sge:        3018326_55     debug sge-j4o0     chew  R       0:20      1 ibc03
DEBUG:sge:        3018326_56     debug sge-j4o0     chew  R       0:20      1 ibc03
DEBUG:sge:        3018326_57     debug sge-j4o0     chew  R       0:20      1 ibc03
DEBUG:sge:        3018326_58     debug sge-j4o0     chew  R       0:20      

DEBUG:sge:        3018326_22     debug sge-j4o0     chew  R       0:30      1 ibc28
DEBUG:sge:        3018326_23     debug sge-j4o0     chew  R       0:30      1 ibc28
DEBUG:sge:        3018326_24     debug sge-j4o0     chew  R       0:30      1 ibc28
DEBUG:sge:        3018326_25     debug sge-j4o0     chew  R       0:30      1 ibc28
DEBUG:sge:        3018326_26     debug sge-j4o0     chew  R       0:30      1 ibc28
DEBUG:sge:        3018326_27     debug sge-j4o0     chew  R       0:30      1 ibc28
DEBUG:sge:        3018326_28     debug sge-j4o0     chew  R       0:30      1 ibc28
DEBUG:sge:        3018326_29     debug sge-j4o0     chew  R       0:30      1 ibc28
DEBUG:sge:        3018326_30     debug sge-j4o0     chew  R       0:30      1 ibc06
DEBUG:sge:        3018326_31     debug sge-j4o0     chew  R       0:30      1 ibc05
DEBUG:sge:        3018326_32     debug sge-j4o0     chew  R       0:30      1 ibc05
DEBUG:sge:        3018326_33     debug sge-j4o0     chew  R       0:30      

...

DEBUG:sge:           2765422     debug plane_25    niina  R 22-22:21:23      1 ibc18
DEBUG:sge:           2765423     debug plane_26    niina  R 22-22:21:23      1 ibc19
DEBUG:sge:           2765424     debug plane_27    niina  R 22-22:21:23      1 ibc19
DEBUG:sge:           2765425     debug plane_28    niina  R 22-22:21:23      1 ibc19
DEBUG:sge:           2765426     debug plane_29    niina  R 22-22:21:23      1 ibc19
DEBUG:sge:           2765427     debug plane_30    niina  R 22-22:21:23      1 ibc19
DEBUG:sge:           2765428     debug plane_31    niina  R 22-22:21:23      1 ibc19
DEBUG:sge:           2765429     debug plane_32    niina  R 22-22:21:23      1 ibc19
DEBUG:sge:           2765430     debug  re01.sh    niina  R 22-21:47:40      1 ibc19
DEBUG:sge:           2765431     debug  re08.sh    niina  R 22-21:47:34      1 ibc19
DEBUG:sge:           2765432     debug  re09.sh    niina  R 22-21:47:31      1 ibc19
DEBUG:sge:           2765433     debug  re14.sh    niina  R 22-21

DEBUG:sge:           2754895     debug job_09.s    niina  R 23-00:51:34      1 ibc01
DEBUG:sge:           2754897     debug job_16.s    niina  R 23-00:51:34      1 ibc01
DEBUG:sge:           2762398     debug plane_01    niina  R 22-22:52:07      1 ibc04
DEBUG:sge:           2762399     debug plane_02    niina  R 22-22:52:07      1 ibc04
DEBUG:sge:           2762400     debug plane_03    niina  R 22-22:52:07      1 ibc06
DEBUG:sge:           2762401     debug plane_04    niina  R 22-22:52:07      1 ibc06
DEBUG:sge:           2762402     debug plane_05    niina  R 22-22:52:07      1 ibc06
DEBUG:sge:           2762403     debug plane_06    niina  R 22-22:52:07      1 ibc19
DEBUG:sge:           2762404     debug plane_07    niina  R 22-22:52:07      1 ibc19
DEBUG:sge:           2762405     debug plane_08    niina  R 22-22:52:07      1 ibc19
DEBUG:sge:           2762406     debug plane_09    niina  R 22-22:52:07      1 ibc18
DEBUG:sge:           2762407     debug plane_10    niina  R 22-22

DEBUG:sge:        3018326_41     debug sge-j4o0     chew  R       2:21      1 ibc04
DEBUG:sge:        3018326_42     debug sge-j4o0     chew  R       2:21      1 ibc04
DEBUG:sge:        3018326_43     debug sge-j4o0     chew  R       2:21      1 ibc04
DEBUG:sge:        3018326_44     debug sge-j4o0     chew  R       2:21      1 ibc04
DEBUG:sge:        3018326_45     debug sge-j4o0     chew  R       2:21      1 ibc04
DEBUG:sge:        3018326_46     debug sge-j4o0     chew  R       2:21      1 ibc04
DEBUG:sge:        3018326_47     debug sge-j4o0     chew  R       2:21      1 ibc04
DEBUG:sge:        3018326_48     debug sge-j4o0     chew  R       2:21      1 ibc04
DEBUG:sge:        3018326_49     debug sge-j4o0     chew  R       2:21      1 ibc04
DEBUG:sge:        3018326_50     debug sge-j4o0     chew  R       2:21      1 ibc04
DEBUG:sge:        3018326_51     debug sge-j4o0     chew  R       2:21      1 ibc04
DEBUG:sge:        3018326_52     debug sge-j4o0     chew  R       2:21      

DEBUG:sge:        3018326_16     debug sge-j4o0     chew  R       2:31      1 ibc28
DEBUG:sge:        3018326_17     debug sge-j4o0     chew  R       2:31      1 ibc28
DEBUG:sge:        3018326_18     debug sge-j4o0     chew  R       2:31      1 ibc28
DEBUG:sge:        3018326_19     debug sge-j4o0     chew  R       2:31      1 ibc28
DEBUG:sge:        3018326_20     debug sge-j4o0     chew  R       2:31      1 ibc28
DEBUG:sge:        3018326_21     debug sge-j4o0     chew  R       2:31      1 ibc28
DEBUG:sge:        3018326_22     debug sge-j4o0     chew  R       2:31      1 ibc28
DEBUG:sge:        3018326_23     debug sge-j4o0     chew  R       2:31      1 ibc28
DEBUG:sge:        3018326_24     debug sge-j4o0     chew  R       2:31      1 ibc28
DEBUG:sge:        3018326_25     debug sge-j4o0     chew  R       2:31      1 ibc28
DEBUG:sge:        3018326_26     debug sge-j4o0     chew  R       2:31      1 ibc28
DEBUG:sge:        3018326_27     debug sge-j4o0     chew  R       2:31      

KeyboardInterrupt: 